In [2]:
import pandas as pd
import re

In [3]:
df = pd.read_csv('./news_data.csv')
nt = pd.read_csv('./nation.csv')


In [4]:
# colums (title, content, wirteda_at, sub_category)
df_ne = df.drop(['news_id','writer','url','main_category','platform'], axis=1)
df_ne = df_ne.dropna()


In [5]:
df_ne['content'] = df_ne['content'].apply(lambda x : re.sub('((\(|\[).*\=.*(\]|\)).*?\= )','',x))  #[서울=뉴시스]신정원 기자 =
df_ne['content'] = df_ne['content'].apply(lambda x : re.sub('\(사진=[^)]+\)','',x)) # (사진=블라블라)
df_ne['content'] = df_ne['content'].apply(lambda x : re.sub('\(현지시간\)','',x)) # (현지시간)
df_ne['content'] = df_ne['content'].apply(lambda x : re.sub('\[|\]|\(|\)','',x)) #[, ], (, )
df_ne['content'] = df_ne['content'].apply(lambda x : re.sub('[가-힣A-z]+\=[가-힣A-z/0-9]+','',x))

#로이터=뉴스1
#뉴델리=연합뉴스
#마닐라=AP/뉴시스


In [6]:
news_list = df_ne.values.tolist()

In [7]:
from summa.summarizer  import summarize
new_sum_list = [news_list[n][1] if summarize(news_list[n][1], words=20) == '' else summarize(news_list[n][1], words=20) for n in range(len(news_list))]
df_ne['content'] = new_sum_list


In [8]:
# df_ne[(df_ne['title'].str.contains('\[사진\]'))&(df_ne['content'].str.len() <= 10)]
# df_ne[(df_ne['title'].str.contains('\[사진\]'))]
# df_ne[((df_ne['content'].str.len() <= 20))]

df_ne = df_ne.drop(df_ne[df_ne['content'].str.len() <= 4].index)
df_ne = df_ne.drop(df_ne[(df_ne['content'].str.contains('파이낸셜뉴스'))].index)
df_ne = df_ne.drop(df_ne[(df_ne['content'].str.contains('블룸버그는'))].index)
df_ne = df_ne.drop(df_ne[(df_ne['content'].str.contains('전진영 기자 jintonic@'))].index)
df_ne = df_ne.drop(df_ne[(df_ne['content'].str.contains('자세한 내용 이어집니다.'))].index)
df_ne = df_ne.drop(df_ne[(df_ne['content'].str.contains('데일리안 = 데스크 by.\n한가마'))].index)
df_ne = df_ne.drop(df_ne[(df_ne['title'].str.contains('홍콩, 하락세 홍콩달러 매수 시장개입..."3개월 만에 6800억원 투입'))].index)
df_ne.reset_index(drop=True, inplace=True)
df_ne.rename(columns={'content':'summa_content'}, inplace=True)

In [10]:
df_ne.to_csv('final_data.csv', index=False)